# Kinematics of the Qubit
**A companion notebook to "Graduate Quantum Information Science"**

In [34]:
import doctest

import numpy as np
from braket.circuits import Circuit, Noise, gates
#from braket.devices import LocalSimulator
from scipy.linalg import expm

# magic word for producing visualizations in notebook
%matplotlib inline

This notebook is meant to illustrate the probability-first approach to quantum used in "Graduate Quantum Information Science". Here we are concerned with the kinematics of probability density matrices in the two outcome situation. The notation and examples largely follow the associated appendix concerning the kinematics of the qubit.

We consider states followed by their transformations. 

### Table of contents
* [Probability density matrices for qubits](#states)
* [Kinematially allowed transformations](#transforms)

# Probability density matrices for qubits<a class="anchor" id="states"></a>

In [2]:
def is_probability_density_matrix(R):
    """Checks if a given array represents a probability density matrix.

    Probability density matrices must be two-dimensional (matrices), with trace
    one and each eigenvalue both real and greater than or equal to zero.

    Args:
        R ([float]): A two-dimensional array of numbers.

    Returns:
        (bool): True if the array represents a probability density vector, False otherwise

    Examples of probability density vectors:

    >>> is_probability_density_matrix([[1, 0],[0, 0]])
    True
    >>> is_probability_density_matrix([[0.5, 0.5], [0.5, 0.5]])
    True
    >>> is_probability_density_matrix(np.array([[1, 0],[0, 0]]))
    True
    >>> is_probability_density_matrix(np.matrix([[1, 0],[0, 0]]))
    True
    >>> is_probability_density_matrix([[.5, -.5], [-.5, .5]])
    True
    >>> is_probability_density_matrix(np.matrix([[0, 0], [0, 1]]))
    True
    >>> is_probability_density_matrix([[0.5, 0],[0, 0.5]])
    True
    >>> is_probability_density_matrix([[0, 0], [0, 1]])
    True
    >>> is_probability_density_matrix([[1 + 0j, 0],[0,0]] )
    True

    Examples of arrays that are not a probability density matrix:

    Not two-dimensional array
    >>> is_probability_density_matrix([1, 0])
    False

    Not all real eigenvalues
    >>> is_probability_density_matrix(np.diag([1 + 1j, 0]))
    False


    Not all positive eigenvalues
    >>> is_probability_density_matrix(np.diag([1.1, -0.1]))
    False
    >>> is_probability_density_matrix(np.diag([-1 + 0j, 0]))
    False

    Not normalized
    >>> is_probability_density_matrix(np.diag([1, 1]))
    False
    """
    # treat m as a numpy matrix, recast input as np.matrix
    R = np.matrix(R)

    # check if one-dimensional
    if not R.shape[0]==R.shape[1]:
        return False


    v = np.linalg.eigvals(R)

    #round off to 14 digits of accuracy

    v = np.round(v*1e14)/1e14

    ## import kinematics_of_prob
    ## return kinematics_of_prob.is_probability_density_vector(v)

    # check if all real
    if not np.isreal(v).all():
        return False
    # check if all positive
    v = np.real(v)
    if (v < -1e-16).any():
        return False
    # check normalization
    if not np.isclose(np.sum(v), 1):
        return False
    return True


#uncomment to run tests:
#doctest.testmod()

In [3]:
Id= np.matrix([[1, 0 ],[ 0, 1]])
X = np.matrix([[0, 1 ],[ 1, 0]])
Y = np.matrix([[0,-1j],[1j, 0]])
Z = np.matrix([[1, 0 ],[ 0,-1]])

In [87]:
def quantum_pdm_from_r(rx,ry,rz):
    # construct a probability density matrix from an input 3-vector
    assert ((rx*rx) + (ry*ry) + (rz*rz)) < 0.5+1e-9
    R = 0.5 * Id + ( rx * X + ry * Y + rz * Z )
    return R

In [94]:
rz=0.75
rx=np.sqrt(1-(0.75*0.75))
rho=quantum_pdm_from_r(rx,0,rz)

print(rz)
print(rx)

print(is_probability_density_matrix(rho))

np.trace(rho@Z)
np.trace(rho@X)

0.75
0.6614378277661477
True


(0.6614378277661477+0j)

In [95]:
.66*.66 + .75*.75

0.9981

# State transformations <a class="anchor" id="transforms"></a>

## Change of basis for a qubit

In [5]:
# Change of basis for a qubit
def is_unitary_cob_matrix(M):
    """
    Checks if a given array represents a unitary change of basis matrix.

    Unitary matrices must be two-dimensional (matrices), such that
    $ U \times U^\dag = \id $

    Args:
        M ([float]): A two-dimensional array of numbers.

    Returns:
        (bool): True if the array represents a unitary matrix, False otherwise

    Examples of unitary matrices:

    >>>  is_unitary_cob_matrix([[1, 0],[0, 1]])
    True
    >>>  is_unitary_cob_matrix([[1, 1], [1, -1]]/np.sqrt(2))
    True
    >>>  is_unitary_cob_matrix(np.matrix([[np.exp(0-7j), 0],[0, np.exp(0+3j)]]))
    True
    >>>  is_unitary_cob_matrix(np.matrix([[np.exp(0-3j), 0],[0, np.exp(0+3j)]]))
    True
    >>>  is_unitary_cob_matrix([[1, 1], [1, -1]]/np.sqrt(2))
    True
    >>>  is_unitary_cob_matrix(np.matrix([[0, 1], [1, 0]]))
    True
    >>>  is_unitary_cob_matrix([[np.cos(3), np.sin(3)], [np.sin(3), -np.cos(3)]])
    True
    >>>  is_unitary_cob_matrix([[np.cos(3), -np.sin(3)], [-np.sin(3), np.cos(3)]])
    True
    >>>  is_unitary_cob_matrix([[1 + 0j, 0],[0,1]] )
    True

    Examples of arrays that are not a probability density matrix:

    Not two-dimensional array
    >>> is_unitary_cob_matrix([1, 0])
    False

    Not normalized
    >>> is_unitary_cob_matrix([[1, 1],[1 -1]]))
    False
    """

    M = np.matrix(M)

    if np.allclose(M @ M.H, np.eye(2)):
        return True
    else:
        return False


In [6]:
def change_of_basis_matrix(theta, phi):
    """Change of basis formula based on Euler ZYZ decomposition"""
    return expm( -1j * Z * theta ) @ expm( -1j * Y * phi )


print(is_unitary_cob_matrix(change_of_basis_matrix(np.random.rand()*2*np.pi, np.random.rand()*np.pi)))


True


In [8]:


#P12 = X
#R1 = P12 @ R @ P12.H
#R2 = P12 @ Rmixed @ P12.H

#print(is_probability_density_matrix(R1))
#print(is_probability_density_matrix(R2))

print(is_unitary_cob_matrix([[np.cos(3),np.sin(3)],[np.sin(3),-np.cos(3)]]))
print(is_unitary_cob_matrix([[np.cos(3),np.sin(3)],[-np.sin(3),np.cos(3)]]))
print(is_unitary_cob_matrix(expm(-1j * np.random.rand() * X)))

True
True
True


## Kraus representation for quantum state transformations

In [9]:
def is_valid_kraus(Ks):
    """Check that the Ks={K_j} jump operators satisfy the Kraus normalization

      Inputs:
        Ks      An array of Kraus operator matrices
        rho     The quantum state the channel should be applied to
    """

    normalization = np.eye(2)*0
    for K in Ks:
        K=np.matrix(K)
        normalization = normalization + K.H @ K

    return np.allclose(normalization,np.eye(2))


In [10]:
def kraus_channel(Ks, rho, TT=False):
    """Make and apply a quantum channel characterized by Ks={K_j} on pdm rho

      Inputs:
         Ks      An array of Kraus operator matrices
         rho     The quantum state the channel should be applied to
         TT      (bool, default False), if true, do superop transpose
    """

    R = id @ rho @ id
    Rf = np.zeros(R.shape)

    for K in Ks:
        K = np.matrix(K)
        if not TT:
            Rf = Rf + K @ R @ K.H
        else:
            Rf = Rf + K.H @ R @ K

    return Rf

**Bi-stochastic transformations**

In [11]:
#np.random.random()
p=0.2513328968202828

Bit_flip_ops = [X*np.sqrt(p) , Id * np.sqrt(1-p)]
Phase_flip_ops = [Z*np.sqrt(p) , Id * np.sqrt(1-p)]
Depol_ops =  [X*np.sqrt(p)/2 , Y*np.sqrt(p)/2, Z*np.sqrt(p)/2, Id * np.sqrt(1-3*p/4)]

def channel_transpose(Ks):
    "Superoperator transpose channel"
    KT=[]
    for K in Ks:
        KT.append(K.H)
    return KT

print(is_valid_kraus(channel_transpose(Bit_flip_ops)))
is_valid_kraus(Bit_flip_ops)



True


True

**General stochastic transformation of a qubit**

In [14]:
#gamma parameter controls the strength of the channel.
#Try playing with this value to see how rapidly states decohere
g=.5

AD = [ np.matrix([[1,0],[0,np.sqrt(1-g)]]) , np.matrix([[0,np.sqrt(g)],[0,0]]) ]


print(is_valid_kraus(AD))
print(is_valid_kraus(channel_transpose(AD)))

True
False


In [46]:
#gamma parameter controls the strength of the channel.
#Try playing with this value
g=1
p=.6
gAD = [
    np.sqrt(p)   *np.matrix([[1,0],[0,np.sqrt(1-g)]]) ,
    np.sqrt(p)   *np.matrix([[0,np.sqrt(g)],[0,0]]) ,
    np.sqrt(1-p) *np.matrix([[0,0],[np.sqrt(g),0]]) ,
    np.sqrt(1-p) *np.matrix([[np.sqrt(1-g),0],[0,1]] )]
print(is_valid_kraus(gAD))

True


## Continuous time transformations

In [76]:
def propagate_rho(H,Ls,rho0,t):
    """Propagate density matrix with vectorized quantum probability balance equation"""

    #vectorize the density matrix
    R0=rho0.flatten()
    R0=R0.T
    
    #Build the generator using the vectorization 
    #eq: vec(ABC) = C^T \otimes A vec(B)
    Gen = -1j*(np.kron(Id,H) - np.kron(H.T,Id))
    for Li in Ls:
        Li=np.matrix(Li)
        Gen = Gen +     np.kron(np.conj(Li),Li)
        Gen = Gen - 0.5*np.kron(Li.T@np.conj(Li),Id)
        Gen = Gen - 0.5*np.kron(Id,Li.H@ Li)
    
    #Do the propagation
    R = expm(Gen*t)@ R0
    
    #reshape and return
    return R.reshape(rho0.shape)

In [86]:
#Markov chain quantum lift

# M is a probability transition matrix
# while W is a rate matrix i.e. W = M/dt

W = np.abs(np.random.randn(2, 2))  # Wij nonzero, no other assumptions
D = np.diag(np.sum(W, axis=0))  # sum along the row, then turn into diagonal matrix

# Here axis=0 or axis=1 will change the stochastic matrix from left to right
# Feel free to test this out

L = W - D
#try rescaling the generator
L = L*4000

print(L)


L00= L[0,0] * np.matrix([[1,0],[0,0]])
L01= L[0,1] * np.matrix([[0,1],[0,0]])
L10= L[1,0] * np.matrix([[0,0],[1,0]])
L11= L[1,1] * np.matrix([[0,0],[0,1]])

Ls = [L00, L01, L10, L11]


rho0=np.matrix([[.75,0],[0,.25]])

H=np.zeros((2,2))


print(propagate_rho(H,Ls,rho0,0))
print(propagate_rho(H,Ls,rho0,1))
print(propagate_rho(H,Ls,rho0,2))
print(propagate_rho(H,Ls,rho0,3))
print(propagate_rho(H,Ls,rho0,4))
print(propagate_rho(H,Ls,rho0,5))

[[ -1013.51474332  12078.17138218]
 [  1013.51474332 -12078.17138218]]
[[0.75+0.j 0.  +0.j]
 [0.  +0.j 0.25+0.j]]
[[0.99300786+0.j 0.        +0.j]
 [0.        +0.j 0.00699215+0.j]]
[[0.99300786+0.j 0.        +0.j]
 [0.        +0.j 0.00699215+0.j]]
[[0.99300785+0.j 0.        +0.j]
 [0.        +0.j 0.00699215+0.j]]
[[0.99300787+0.j 0.        +0.j]
 [0.        +0.j 0.00699215+0.j]]
[[0.99300785+0.j 0.        +0.j]
 [0.        +0.j 0.00699215+0.j]]


In [75]:
#Amplitude dampening channel in continuous time
p=.6
g=1
Ls = [
    np.sqrt(p)   *np.matrix([[1,0],[0,np.sqrt(1-g)]]) ,
    np.sqrt(p)   *np.matrix([[0,np.sqrt(g)],[0,0]]) ,
    np.sqrt(1-p) *np.matrix([[0,0],[np.sqrt(g),0]]) ,
    np.sqrt(1-p) *np.matrix([[np.sqrt(1-g),0],[0,1]] )]

H=np.zeros((2,2))
rho0=np.matrix([[.75,0],[0,.25]])
               
print(propagate_rho(H,Ls,rho0,0))
print(propagate_rho(H,Ls,rho0,1))
print(propagate_rho(H,Ls,rho0,2))   
print(propagate_rho(H,Ls,rho0,3))   
print(propagate_rho(H,Ls,rho0,4))  
print(propagate_rho(H,Ls,rho0,5))  

[[0.75+0.j 0.  +0.j]
 [0.  +0.j 0.25+0.j]]
[[0.65518192+0.j 0.        +0.j]
 [0.        +0.j 0.34481808+0.j]]
[[0.62030029+0.j 0.        +0.j]
 [0.        +0.j 0.37969971+0.j]]
[[0.60746806+0.j 0.        +0.j]
 [0.        +0.j 0.39253194+0.j]]
[[0.60274735+0.j 0.        +0.j]
 [0.        +0.j 0.39725265+0.j]]
[[0.60101069+0.j 0.        +0.j]
 [0.        +0.j 0.39898931+0.j]]


In [59]:
#Schrodinger equation

Ls=[np.zeros((2,2))]

#try rescaling H
H = 2*Y

#pure state
rho0=np.matrix([[1,0],[0,0]])


print(propagate_rho(H,Ls,rho0,0))
print(propagate_rho(H,Ls,rho0,.1))
print(propagate_rho(H,Ls,rho0,.2))   
print(propagate_rho(H,Ls,rho0,.3))   
print(propagate_rho(H,Ls,rho0,.4))  
print(propagate_rho(H,Ls,rho0,.5))  
print(propagate_rho(H,Ls,rho0,.6))  
print(propagate_rho(H,Ls,rho0,.7))  
print(propagate_rho(H,Ls,rho0,.8))  
print(propagate_rho(H,Ls,rho0,.9))  
print(propagate_rho(H,Ls,rho0,1))  

#plot the [0,0] component of the probability density matrix

[[1.+0.j 0.+0.j]
 [0.+0.j 0.+0.j]]
[[0.9605305 +0.j 0.19470917+0.j]
 [0.19470917+0.j 0.0394695 +0.j]]
[[0.84835335+0.j 0.35867805+0.j]
 [0.35867805+0.j 0.15164665+0.j]]
[[0.68117888+0.j 0.46601954+0.j]
 [0.46601954+0.j 0.31882112+0.j]]
[[0.48540024+0.j 0.4997868 +0.j]
 [0.4997868 +0.j 0.51459976+0.j]]
[[0.29192658+0.j 0.45464871+0.j]
 [0.45464871+0.j 0.70807342+0.j]]
[[0.13130314+0.j 0.33773159+0.j]
 [0.33773159+0.j 0.86869686+0.j]]
[[0.02888883+0.j 0.16749408+0.j]
 [0.16749408+0.j 0.97111117+0.j]]
[[ 8.52612103e-04+0.j -2.91870717e-02+0.j]
 [-2.91870717e-02+0.j  9.99147388e-01+0.j]]
[[ 0.05162079+0.j -0.22126022+0.j]
 [-0.22126022+0.j  0.94837921+0.j]]
[[ 0.17317819+0.j -0.37840125+0.j]
 [-0.37840125+0.j  0.82682181+0.j]]


In [1]:
# JDWhitfield 2024

In [18]:
# Turn our probability p (set at the top) into a probability density vector
vecp = np.matrix([[p], [1 - p]])
print(f"vecp: \n{vecp}")
# vecp and its transpose should both be probability density vectors
assert is_probability_density_vector(vecp)
assert is_probability_density_vector(vecp.T)

# bias parameter
b = p - 1 / 2

vecp: 
[[0.5]
 [0.5]]


NameError: name 'is_probability_density_vector' is not defined

In [ ]:
# Useful matrices to represent change of basis of 2D probability vectors
I = np.eye(2)  # the identity
P12 = np.matrix([[0, 1], [1, 0]])  # permutation (1 2) -- change of basis for a bit

In [ ]:
# bistochastic change of basis for a bit

# NOTE: the ": float" is a type hint.
# Using them is *not* necessary, but they can help document what your function does
# (and check that what you've done is correct!).
def change_of_basis_matrix_2d(w: float):
    """Given a weight w, returns a change of basis matrix for 2D probability vectors

    Args:
        w (float): The weight given to permutation P12 (in the range [0, 1])

    Returns:
        M (np.matrix): Two [2x2] matrices representing a bistochastic channel

    Raises:
        ValueError: if the weight w is not real and in the range [0, 1]

    Examples:
    >>> change_of_basis_matrix_2d(0)
    matrix([[1., 0.],
            [0., 1.]])

    >>> change_of_basis_matrix_2d(1)
    matrix([[0., 1.],
            [1., 0.]])

    >>> change_of_basis_matrix_2d(0.5)
    matrix([[0.5, 0.5],
            [0.5, 0.5]])

    >>> change_of_basis_matrix_2d(0+0j)
    matrix([[1., 0.],
            [0., 1.]])

    >>> change_of_basis_matrix_2d(-1)
    Traceback (most recent call last):
    ...
    ValueError: w must be between 0 and 1 (inclusive), given: -1

    >>> change_of_basis_matrix_2d(0+1j)
    Traceback (most recent call last):
    ...
    ValueError: w must be real, given: 1j

    """
    if not np.isreal(w):
        raise ValueError(f"w must be real, given: {w}")
    w = np.real(w)  # remove "+0j" imaginary component
    if w < 0 or w > 1:
        raise ValueError(f"w must be between 0 and 1 (inclusive), given: {w}")
    M = w * P12 + (1 - w) * I
    return M


# uncomment to run tests:
# doctest.testmod()

In [ ]:
assert is_probability_density_vector(vecp)
# NOTE: the "@" symbol is how numpy expresses matrix multiplication
assert is_probability_density_vector(change_of_basis_matrix_2d(0) @ vecp)
assert is_probability_density_vector(change_of_basis_matrix_2d(1) @ vecp)
assert is_probability_density_vector(change_of_basis_matrix_2d(0.5) @ vecp)

In [ ]:
# general stochastic transformations of a bit
def general_markov_transform_2d(w1: float, w2: float):
    """Given two weights w1 and w2, return the general (2D) Markov transform

    Args:
        w1 (float): The weight of no permutation (in the range [0, 1])
        w2 (float): The weight of no permutation (in the range [0, 1])

    Returns:
        M (np.matrix): A 2x2 matrix representing a general Markov transform

    Raises:
        ValueError: if the weights (w1 and w2) are not both real and in the range [0, 1]

    Examples:

    >>> general_markov_transform_2d(0, 0)
    matrix([[0., 0.],
            [1., 1.]])

    >>> general_markov_transform_2d(0, 1)
    matrix([[0., 1.],
            [1., 0.]])

    >>> general_markov_transform_2d(0.5, 0.5)
    matrix([[0.5, 0.5],
            [0.5, 0.5]])

    Failing examples:

    >>> general_markov_transform_2d(0+1j, 0)
    Traceback (most recent call last):
    ...
    ValueError: w1 must be real, given: 1j

    >>> general_markov_transform_2d(0, 0+1j)
    Traceback (most recent call last):
    ...
    ValueError: w2 must be real, given: 1j

    >>> general_markov_transform_2d(2, 0)
    Traceback (most recent call last):
    ...
    ValueError: w1 must be between 0 and 1 (inclusive), given: 2

    >>> general_markov_transform_2d(0, 2)
    Traceback (most recent call last):
    ...
    ValueError: w2 must be between 0 and 1 (inclusive), given: 2
    """
    # w1 checks
    if not np.isreal(w1):
        raise ValueError(f"w1 must be real, given: {w1}")
    w1 = np.real(w1)  # remove "+0j" imaginary component
    if w1 < 0 or w1 > 1:
        raise ValueError(f"w1 must be between 0 and 1 (inclusive), given: {w1}")
    # w2 checks
    if not np.isreal(w2):
        raise ValueError(f"w2 must be real, given: {w2}")
    w2 = np.real(w2)  # remove "+0j" imaginary component
    if w2 < 0 or w2 > 1:
        raise ValueError(f"w2 must be between 0 and 1 (inclusive), given: {w2}")

    M = np.matrix([[w1, w2], [1.0 - w1, 1.0 - w2]])
    return M


# uncomment to run tests:
# doctest.testmod()

In [ ]:
assert is_probability_density_vector(general_markov_transform_2d(0, 0) @ vecp)
assert is_probability_density_vector(general_markov_transform_2d(1, 0) @ vecp)
assert is_probability_density_vector(general_markov_transform_2d(0, 1) @ vecp)
assert is_probability_density_vector(general_markov_transform_2d(0.5, 0.5) @ vecp)

In [ ]:
# M is a probability transition matrix
# while W is a rate matrix i.e. W = M/dt

W = np.abs(np.random.randn(2, 2))  # Wij nonzero, no other assumptions
D = np.diag(np.sum(W, axis=0))  # sum along the row, then turn into diagonal matrix

# Here axis=0 or axis=1 will change the stochastic matrix from left to right
# Feel free to test this out

L = W - D

print(f"L generates changes in time: \n{L}")

# Students: Play with this value! Does it have to be positive?
t = 1.0

print(f"vecp is a probability density vector")
print(is_probability_density_vector(vecp))

print(f"L generates changes in time on the right")
print(is_probability_density_vector(expm(L * t) @ vecp))

print(f"But not on the left")
print(is_probability_density_vector(vecp.T * expm(L * t)))

print(f"Its transpose doesn't work on the right")
print(is_probability_density_vector(expm(L.T * t) @ vecp))

print(f"But it does work on the left")
print(is_probability_density_vector((vecp.T) @ expm(L.T * t)))